<a href="https://colab.research.google.com/github/mtzig/LIDC_GDRO/blob/main/notebooks/lidc_cnn_ERM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ERM CNN Model for Malignancy

#First We setup the repo

In [4]:
# Only run if on Colab

!git clone https://github.com/mtzig/LIDC_GDRO.git
%cd /content/LIDC_GDRO

Cloning into 'LIDC_GDRO'...
remote: Enumerating objects: 3040, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 3040 (delta 61), reused 77 (delta 33), pack-reused 2925
Receiving objects: 100% (3040/3040), 39.77 MiB | 27.41 MiB/s, done.
Resolving deltas: 100% (2873/2873), done.
Checking out files: 100% (5384/5384), done.
/content/LIDC_GDRO


In [ ]:
#!git pull

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import NoduleDataset
from dataloaders import SubtypedDataLoader

In [6]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Using cpu")
    DEVICE = torch.device("cpu")

Using cpu


#Next We get our data